---
# 올리브영 스킨/로션 제품군 크롤링
* "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010013&fltDispCatNo=&prdSort=01&pageIdx=3"
---

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# 전체 페이지

In [5]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service,options=options)


time.sleep(0.2)

prd_items = []
prd_brands = []
prd_names = []
prd_prices = []
prd_infos = []
prd_thumbs = [] 
try: 
    for page in range(1,3):
        url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010013&fltDispCatNo=&prdSort=01&pageIdx={page}"
        driver.get(url)

        time.sleep(1.0)
        prd_list = driver.find_element(By.ID,"Contents")
        # prd = prd_list.find_element(By.CLASS_NAME,"cate_prd_list")      # 한 페이지에 이 클래스 이름을 가진 ul이 여러 개고 그 ul마다 각각 상품 네개씩 있음
        prd = prd_list.find_elements(By.CLASS_NAME,"prd_info")              # 그냥 바로 상품 각각 찾기

        for i in range(len(prd)):
            prds = driver.find_elements(By.CLASS_NAME, "prd_info")  # Refresh the list of prd elements
            each_prd = prds[i]
            each_prd.click()

            info_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "buyInfo")))
            info_btn.click()

            prd_items = driver.find_element(By.CLASS_NAME, "right_area")
            prd_brands.append(prd_items.find_element(By.ID, "moveBrandShop").text)    
            prd_name = prd_items.find_element(By.CLASS_NAME, "prd_name").text
            prd_names.append(prd_name)
            prd_prices.append(prd_items.find_element(By.CLASS_NAME, "price").text.split('\n')[0])
            
            
            prd_images_area = driver.find_element(By.ID, "prd_thumb_list")
            thumbs = prd_images_area.find_elements(By.TAG_NAME, "li")
            imageNames = []
            for idx, li in enumerate(thumbs):
                
                imgs = li.find_element(By.TAG_NAME, "img")
                imgsrc = imgs.get_attribute("src")
                imgres = requests.get(imgsrc)

                imageName = f"./prdimgs/page_{page}_prd_{i}_thumb{idx + 1}.jpg"
                with open(imageName, "wb") as f:
                    f.write(imgres.content)
                imageNames.append(imageName)
                
            prd_thumbs.append(imageNames)
            prd_info = WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.ID, "artcInfo")))
            prd_infos.append(prd_info.text.split("\n"))
            driver.back()
            time.sleep(0.2)
finally:
    driver.quit()

In [6]:
# page=2까지
import pandas as pd
skin = pd.DataFrame({"제품명":prd_names, "브랜드":prd_brands, "가격":prd_prices,'상품정보':prd_infos,'썸네일':prd_thumbs})
skin

,제품명,브랜드,가격,상품정보,썸네일
0,라운드랩 자작나무 수분 토너 1+1 기획 (300ml+300ml),라운드랩,"24,000","[내용물의 용량 또는 중량 300ml*2ea, 제품 주요 사양 모든 피부용, 사용기...","[./prdimgs/page_1_prd_0_thumb1.jpg, ./prdimgs/..."
1,[1월올영PICK/1+1] 라네즈 크림스킨 170ml 리필 기획 (+170ml 리필),라네즈,"33,000","[내용물의 용량 또는 중량 170ml (+170ml 증정), 제품 주요 사양 모든피...",[./prdimgs/page_1_prd_1_thumb1.jpg]
2,[2023어워즈/3년연속] 아누아 어성초 77 수딩 토너 350ml 어워즈 한정 기...,아누아,"45,000",[내용물의 용량 또는 중량 토너 본품 350ml+토너 리필 350ml+진정마스크 1...,"[./prdimgs/page_1_prd_2_thumb1.jpg, ./prdimgs/..."
3,[1월 올영픽/대용량기획] 토리든 다이브인 저분자 히알루론산 토너 500ml 기획 ...,토리든,"29,000",[내용물의 용량 또는 중량 [본품] 다이브인 토너 500ml [증정] 화장솜 60매...,"[./prdimgs/page_1_prd_3_thumb1.jpg, ./prdimgs/..."
4,[2023어워즈] 아비브 어성초 카밍 토너 스킨부스터 더블 기획 (200ml+200ml),아비브,"39,000","[내용물의 용량 또는 중량 200ml+200ml, 제품 주요 사양 모든 피부용, 사...","[./prdimgs/page_1_prd_4_thumb1.jpg, ./prdimgs/..."
5,라운드랩 1025 독도 토너 (본품200ml+50ml 추가 증정),라운드랩,"15,000",[내용물의 용량 또는 중량 [본품] 1025 독도 토너 200ml + [증정품] 1...,"[./prdimgs/page_1_prd_5_thumb1.jpg, ./prdimgs/..."
6,[NEW]브링그린 티트리시카수딩토너 500mL한정기획(+100mL증정),브링그린,"27,000","[내용물의 용량 또는 중량 ■ 토너 500mL+100mL, 제품 주요 사양 ■ 모든...","[./prdimgs/page_1_prd_6_thumb1.jpg, ./prdimgs/..."
7,[화해1위] 에스네이처 아쿠아 오아시스 토너 300ml 기획(+ 수분크림30ml증정),에스네이처,"24,000","[내용물의 용량 또는 중량 토너 300ml, 크림 30ml, 제품 주요 사양 모든 ...","[./prdimgs/page_1_prd_7_thumb1.jpg, ./prdimgs/..."
8,[탄력광채] 넘버즈인 3번 결광가득 에센스 토너 300ml 대용량 기획,넘버즈인,"32,000","[내용물의 용량 또는 중량 300ml, 제품 주요 사양 모든 피부 용, 사용기한(또...","[./prdimgs/page_1_prd_8_thumb1.jpg, ./prdimgs/..."
9,[단독기획] 토니모리 원더 세라마이드 모찌 토너 (본품500ml+100ml),토니모리,"15,000","[내용물의 용량 또는 중량 500ml+100ml, 제품 주요 사양 모든 피부 타입,...","[./prdimgs/page_1_prd_9_thumb1.jpg, ./prdimgs/..."


In [7]:
skin.to_csv('./output/skin_items.csv',index=False)

In [8]:
skin.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     48 non-null     object
 1   브랜드     48 non-null     object
 2   가격      48 non-null     object
 3   상품정보    48 non-null     object
 4   썸네일     48 non-null     object
dtypes: object(5)
memory usage: 2.0+ KB
